In [1]:
# complete_training.py
import os
import random
from pathlib import Path
import math
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import time
import json
import gc

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import transforms, models, datasets
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Memory optimization
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Setup
def setup_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        torch.backends.cudnn.benchmark = True
        print("✅ GPU enabled")
    else:
        device = torch.device("cpu")
        print("❌ Using CPU")
    return device

DEVICE = setup_device()

# Configuration
BATCH_SIZE = 16  # Safe for 8GB GPU
EPOCHS = 50
IMAGE_SIZE = 224
LEARNING_RATE = 5e-4
OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

# Dataset paths
ARCHIVE_DIR = Path(r"C:\Users\shiva\Downloads\brain tumor dataset")
TRAIN_DIR = ARCHIVE_DIR / "Training"
VAL_DIR = ARCHIVE_DIR / "Testing"

print("🔍 Checking dataset paths...")
print(f"Training: {TRAIN_DIR.exists()}, Validation: {VAL_DIR.exists()}")

# Data transforms
def get_transforms():
    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(IMAGE_SIZE),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(0.2, 0.2, 0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    val_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    return train_transform, val_transform

# Create datasets
train_transform, val_transform = get_transforms()
train_ds = datasets.ImageFolder(TRAIN_DIR, transform=train_transform)
val_ds = datasets.ImageFolder(VAL_DIR, transform=val_transform)

NUM_CLASSES = len(train_ds.classes)
classes = train_ds.classes

print(f"✅ Datasets loaded: {len(train_ds)} train, {len(val_ds)} val, {NUM_CLASSES} classes")

# Create dataloaders
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Save class info
class_info = {"classes": classes, "num_classes": NUM_CLASSES}
with open(OUTPUT_DIR / "class_info.json", 'w') as f:
    json.dump(class_info, f, indent=2)

# Model - Using the proven ResNet50 architecture
def create_model(num_classes):
    print("🔄 Creating ResNet50 model...")
    model = models.resnet50(pretrained=True)
    
    # Replace classifier
    in_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(0.2),
        nn.Linear(in_features, 512),
        nn.ReLU(),
        nn.BatchNorm1d(512),
        nn.Dropout(0.2),
        nn.Linear(512, num_classes)
    )
    
    model = model.to(DEVICE)
    print(f"✅ Model created with {sum(p.numel() for p in model.parameters()):,} parameters")
    return model

model = create_model(NUM_CLASSES)

# Loss function with class weights
class_counts = np.bincount([label for _, label in train_ds.samples])
class_weights = 1.0 / class_counts
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(DEVICE)

criterion = nn.CrossEntropyLoss(weight=class_weights)
print("✅ Loss function with class weights created")

# Optimizer and scheduler
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

# Mixed precision
scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None

# Training functions
def train_epoch(model, loader, criterion, optimizer, scaler, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(loader, desc="🚀 Training")
    for inputs, labels in pbar:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        if scaler:
            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        pbar.set_postfix({
            'Loss': f'{running_loss/(total//BATCH_SIZE+1):.3f}',
            'Acc': f'{100.*correct/total:.2f}%'
        })
    
    return running_loss/len(loader), 100.*correct/total

def validate_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in tqdm(loader, desc="🧪 Validation"):
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return running_loss/len(loader), 100.*correct/total

# Early stopping
class EarlyStopping:
    def __init__(self, patience=10, min_delta=1e-4):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
    
    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

early_stopper = EarlyStopping(patience=10)

# Save function for multiple formats
def save_model_multiple_formats(model, classes, filename, epoch=None, val_acc=None):
    """Save model in .h5, .keras, .legacy formats"""
    base_path = OUTPUT_DIR / filename
    
    # Save as .h5
    torch.save(model.state_dict(), f"{base_path}.h5")
    
    # Save as .keras  
    torch.save(model.state_dict(), f"{base_path}.keras")
    
    # Save as .legacy with metadata
    legacy_data = {
        'model_state_dict': model.state_dict(),
        'classes': classes,
        'epoch': epoch,
        'val_acc': val_acc,
        'model_type': 'resnet50'
    }
    torch.save(legacy_data, f"{base_path}.legacy")
    
    print(f"💾 Model saved as {filename}.h5, .keras, .legacy")

# Training history
history = {
    'train_loss': [], 'val_loss': [],
    'train_acc': [], 'val_acc': []
}

print("🚀 STARTING TRAINING...")

best_val_acc = 0
for epoch in range(EPOCHS):
    print(f"\n{'='*50}")
    print(f"🎯 Epoch {epoch+1}/{EPOCHS}")
    print(f"{'='*50}")
    
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, scaler, DEVICE)
    
    # Validate
    val_loss, val_acc = validate_epoch(model, val_loader, criterion, DEVICE)
    
    # Update history
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['train_acc'].append(train_acc)
    history['val_acc'].append(val_acc)
    
    print(f"📊 Train Loss: {train_loss:.4f}, Acc: {train_acc:.2f}%")
    print(f"📊 Val Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%")
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        save_model_multiple_formats(model, classes, "best_model", epoch+1, val_acc)
        print(f"🎯 New best validation accuracy: {val_acc:.2f}%")
    
    # Save checkpoint
    save_model_multiple_formats(model, classes, "last_checkpoint", epoch+1, val_acc)
    
    # Early stopping
    early_stopper(val_loss)
    if early_stopper.early_stop:
        print("🛑 Early stopping triggered")
        break
    
    # Step scheduler
    scheduler.step()

# Save final model
save_model_multiple_formats(model, classes, "final_model", EPOCHS, best_val_acc)

# Save training history and info
training_info = {
    'best_val_accuracy': best_val_acc,
    'total_epochs': epoch + 1,
    'classes': classes,
    'num_classes': NUM_CLASSES,
    'training_history': history
}

with open(OUTPUT_DIR / "training_info.json", 'w') as f:
    json.dump(training_info, f, indent=2)

print(f"\n🎉 TRAINING COMPLETED!")
print(f"🏆 Best validation accuracy: {best_val_acc:.2f}%")
print(f"💾 Models saved in outputs/ directory:")
print(f"   - best_model.h5, .keras, .legacy")
print(f"   - final_model.h5, .keras, .legacy") 
print(f"   - training_info.json")

✅ GPU enabled
🔍 Checking dataset paths...
Training: True, Validation: True
✅ Datasets loaded: 5712 train, 1311 val, 4 classes
🔄 Creating ResNet50 model...


C:\Users\shiva\Downloads\fakeimgenv311\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\shiva\Downloads\fakeimgenv311\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\shiva\Downloads\fakeimgenv311\lib\site-packages\torch\nn\modules\module.py:1326: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(
C:\Users\shiva\AppData\Local\Temp\ipykernel_4720\

✅ Model created with 24,560,196 parameters
✅ Loss function with class weights created
🚀 STARTING TRAINING...

🎯 Epoch 1/50


🚀 Training:   0%|                                                                             | 0/357 [00:00<?, ?it/s]C:\Users\shiva\AppData\Local\Temp\ipykernel_4720\933839749.py:141: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
🧪 Validation: 100%|███████████████████████████████████████████████████████████████████| 82/82 [00:14<00:00,  5.60it/s]


📊 Train Loss: 0.6204, Acc: 76.94%
📊 Val Loss: 0.4628, Acc: 81.01%
💾 Model saved as best_model.h5, .keras, .legacy
🎯 New best validation accuracy: 81.01%
💾 Model saved as last_checkpoint.h5, .keras, .legacy

🎯 Epoch 2/50


...

📊 Train Loss: 0.0806, Acc: 97.15%
📊 Val Loss: 0.0280, Acc: 99.39%
💾 Model saved as last_checkpoint.h5, .keras, .legacy

🎯 Epoch 48/50


🧪 Validation: 100%|███████████████████████████████████████████████████████████████████| 82/82 [00:11<00:00,  6.98it/s]


📊 Train Loss: 0.0856, Acc: 97.13%
📊 Val Loss: 0.0275, Acc: 99.24%
💾 Model saved as last_checkpoint.h5, .keras, .legacy

🎯 Epoch 49/50


🧪 Validation: 100%|███████████████████████████████████████████████████████████████████| 82/82 [00:12<00:00,  6.82it/s]


📊 Train Loss: 0.0755, Acc: 97.50%
📊 Val Loss: 0.0285, Acc: 99.39%
💾 Model saved as last_checkpoint.h5, .keras, .legacy
🛑 Early stopping triggered
💾 Model saved as final_model.h5, .keras, .legacy

🎉 TRAINING COMPLETED!
🏆 Best validation accuracy: 99.47%
💾 Models saved in outputs/ directory:
   - best_model.h5, .keras, .legacy
   - final_model.h5, .keras, .legacy
   - training_info.json
